In [1]:
# import libraries
import numpy as np
import pandas as pd
import random
import io
import requests
from datetime import datetime, timezone, timedelta
import datetime as dt
import time

import six
from six import StringIO
import sys
from sklearn.tree import export_graphviz
#from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

#for showing 2D plot
%matplotlib inline 

# to be able to see multiple ouputs from sungle cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Step 1. Import sample data

In [2]:
# import sample data - flights
file_path = '../data/samples_1000/flights1000.csv'
flights1000 = pd.read_csv(file_path)
print('sample data: flights')
flights1000.info()
flights1000.head()

file_path2 = '../data/samples_1000/flights_test1000.csv'
flights_test1000 = pd.read_csv(file_path2)
print('sample data: flights_test')
flights_test1000.info()
flights_test1000.head()

file_path3 = '../data/samples_1000/fuel_consumption1000.csv'
fuel_cons1000 = pd.read_csv(file_path3)
print('sample data: fuel_consumption')
fuel_cons1000.info()
fuel_cons1000.head()

file_path4 = '../data/samples_1000/passengers1000.csv'
passengers1000 = pd.read_csv(file_path4)
print('sample data: passengers')
passengers1000.info()
passengers1000.head()

sample data: flights
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 42 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   fl_date              1000 non-null   object 
 1   mkt_unique_carrier   1000 non-null   object 
 2   branded_code_share   1000 non-null   object 
 3   mkt_carrier          1000 non-null   object 
 4   mkt_carrier_fl_num   1000 non-null   int64  
 5   op_unique_carrier    1000 non-null   object 
 6   tail_num             1000 non-null   object 
 7   op_carrier_fl_num    1000 non-null   int64  
 8   origin_airport_id    1000 non-null   int64  
 9   origin               1000 non-null   object 
 10  origin_city_name     1000 non-null   object 
 11  dest_airport_id      1000 non-null   int64  
 12  dest                 1000 non-null   object 
 13  dest_city_name       1000 non-null   object 
 14  crs_dep_time         1000 non-null   int64  
 15  dep_time          

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-12-25,AA,AA_CODESHARE,AA,5178,OH,N609NN,5178,11057,CLT,...,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-25,AA,AA_CODESHARE,AA,5178,OH,N609NN,5178,11996,GSP,...,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-12-25,AA,AA_CODESHARE,AA,5180,OH,N527EA,5180,11057,CLT,...,564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-12-25,AA,AA_CODESHARE,AA,5180,OH,N527EA,5180,12197,HPN,...,564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-25,AA,AA_CODESHARE,AA,5182,OH,N548NN,5182,11267,DAY,...,370,2.0,0.0,1.0,0.0,64.0,NaN,NaN,NaN,NaN


sample data: flights_test
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   fl_date             1000 non-null   int64 
 1   mkt_unique_carrier  1000 non-null   object
 2   branded_code_share  1000 non-null   object
 3   mkt_carrier         1000 non-null   object
 4   mkt_carrier_fl_num  1000 non-null   int64 
 5   op_unique_carrier   1000 non-null   object
 6   tail_num            1000 non-null   object
 7   op_carrier_fl_num   1000 non-null   int64 
 8   origin_airport_id   1000 non-null   int64 
 9   origin              1000 non-null   object
 10  origin_city_name    1000 non-null   object
 11  dest_airport_id     1000 non-null   int64 
 12  dest                1000 non-null   object
 13  dest_city_name      1000 non-null   object
 14  crs_dep_time        1000 non-null   int64 
 15  crs_arr_time        1000 non-null   int64 
 16 

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,1577865600000,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,1577865600000,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,1577865600000,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,1577865600000,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,1577865600000,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


sample data: fuel_consumption
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   month              1000 non-null   int64  
 1   airline_id         999 non-null    float64
 2   unique_carrier     999 non-null    object 
 3   carrier            1000 non-null   object 
 4   carrier_name       999 non-null    object 
 5   carrier_group_new  1000 non-null   int64  
 6   sdomt_gallons      1000 non-null   int64  
 7   satl_gallons       1000 non-null   int64  
 8   spac_gallons       1000 non-null   int64  
 9   slat_gallons       1000 non-null   int64  
 10  sint_gallons       1000 non-null   int64  
 11  ts_gallons         1000 non-null   int64  
 12  tdomt_gallons      1000 non-null   int64  
 13  tint_gallons       1000 non-null   int64  
 14  total_gallons      1000 non-null   int64  
 15  sdomt_cost         1000 non-null   int64  


,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,NaN,NaN,0JQ,NaN,1,0,0,0,0,...,0,0,0,0,0,0,5100,0,5100,2016
1,1,21352.0,0WQ,0WQ,Avjet Corporation,1,0,0,0,0,...,0,0,0,0,0,0,396216,140239,536455,2016
2,1,21645.0,23Q,23Q,Songbird Airways Inc.,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016
3,1,21652.0,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016
4,1,20408.0,5V,5V,Tatonduk Outfitters Limited d/b/a Everts Air A...,1,260848,0,0,0,...,522405,0,0,0,0,522405,569497,0,569497,2016


sample data: passengers
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   departures_scheduled   1000 non-null   int64 
 1   departures_performed   1000 non-null   int64 
 2   payload                1000 non-null   int64 
 3   seats                  1000 non-null   int64 
 4   passengers             1000 non-null   int64 
 5   freight                1000 non-null   int64 
 6   mail                   1000 non-null   int64 
 7   distance               1000 non-null   int64 
 8   ramp_to_ramp           1000 non-null   int64 
 9   air_time               1000 non-null   int64 
 10  unique_carrier         1000 non-null   object
 11  airline_id             1000 non-null   int64 
 12  unique_carrier_name    1000 non-null   object
 13  region                 1000 non-null   object
 14  carrier                1000 non-null   object
 15

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,0,1,3300,9,0,0,0,18,19,13,...,US,United States,4,416,1,2015,5,1,L,DU
1,0,2,11200,0,0,3309,2325,21,35,23,...,US,United States,4,412,2,2015,5,1,G,DU
2,0,1,5600,0,0,0,1416,51,28,22,...,US,United States,4,412,2,2015,5,1,G,DU
3,0,2,11200,0,0,0,0,109,96,84,...,US,United States,4,412,2,2015,5,1,G,DU
4,0,3,16800,0,0,589,3666,71,97,79,...,US,United States,4,412,2,2015,5,1,G,DU


### Step 2. Draw Database Diagram

In [3]:
# function that return db table from data dataframe:
def get_dbtable(df):
    colname = df.columns.tolist()
    non_na = df.count()
    nullcount = df.isnull().sum()
    dtypes = df.dtypes

    df_table = pd.DataFrame({'columns': colname, 
                                  'Non-Null Count': non_na,
                                  'Null Count': nullcount,
                                  'Dtype': dtypes})

    # remove own index with default index
    df_table.reset_index(inplace = True, drop = True)
    
    return df_table

In [4]:
fl_table = get_dbtable(flights1000)
fl_test_table = get_dbtable(flights_test1000)
fuel_table = get_dbtable(fuel_cons1000)
passengr_table = get_dbtable(passengers1000)

fl_table
fl_test_table
fuel_table
passengr_table

,columns,Non-Null Count,Null Count,Dtype
0,fl_date,1000,0,object
1,mkt_unique_carrier,1000,0,object
2,branded_code_share,1000,0,object
3,mkt_carrier,1000,0,object
4,mkt_carrier_fl_num,1000,0,int64
5,op_unique_carrier,1000,0,object
6,tail_num,1000,0,object
7,op_carrier_fl_num,1000,0,int64
8,origin_airport_id,1000,0,int64
9,origin,1000,0,object


,columns,Non-Null Count,Null Count,Dtype
0,fl_date,1000,0,int64
1,mkt_unique_carrier,1000,0,object
2,branded_code_share,1000,0,object
3,mkt_carrier,1000,0,object
4,mkt_carrier_fl_num,1000,0,int64
5,op_unique_carrier,1000,0,object
6,tail_num,1000,0,object
7,op_carrier_fl_num,1000,0,int64
8,origin_airport_id,1000,0,int64
9,origin,1000,0,object


,columns,Non-Null Count,Null Count,Dtype
0,month,1000,0,int64
1,airline_id,999,1,float64
2,unique_carrier,999,1,object
3,carrier,1000,0,object
4,carrier_name,999,1,object
5,carrier_group_new,1000,0,int64
6,sdomt_gallons,1000,0,int64
7,satl_gallons,1000,0,int64
8,spac_gallons,1000,0,int64
9,slat_gallons,1000,0,int64


,columns,Non-Null Count,Null Count,Dtype
0,departures_scheduled,1000,0,int64
1,departures_performed,1000,0,int64
2,payload,1000,0,int64
3,seats,1000,0,int64
4,passengers,1000,0,int64
5,freight,1000,0,int64
6,mail,1000,0,int64
7,distance,1000,0,int64
8,ramp_to_ramp,1000,0,int64
9,air_time,1000,0,int64


### __Obtain the original datasets from SQL Satabase__
> - Connect & access the Pregsql through Python
> - Extract the data from the Dababase Table
> - Transform SQL Database to Pandas DataFrame
> - Save the data to csv
> - https://gist.github.com/00krishna/9026574

#### __Download Testing Datatset: flights_test__

In [5]:
# how to get pandas data from postgree sql using python 
# pip or conda to install SQLAlchemy
import psycopg2 as pg
import pandas.io.sql as psql

connection = pg.connect("host='mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com' dbname=mid_term_project user=lhl_student password='lhl_student'")
test_df = pd.read_sql_query('select * from flights_test',con=connection)

test_df.info()
test_df.head()

C:\Users\User\anaconda3\envs\jupyterlab-3.2.1\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660556 entries, 0 to 660555
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   fl_date             660556 non-null  datetime64[ns]
 1   mkt_unique_carrier  660556 non-null  object        
 2   branded_code_share  660556 non-null  object        
 3   mkt_carrier         660556 non-null  object        
 4   mkt_carrier_fl_num  660556 non-null  int64         
 5   op_unique_carrier   660556 non-null  object        
 6   tail_num            659057 non-null  object        
 7   op_carrier_fl_num   660556 non-null  int64         
 8   origin_airport_id   660556 non-null  int64         
 9   origin              660556 non-null  object        
 10  origin_city_name    660556 non-null  object        
 11  dest_airport_id     660556 non-null  int64         
 12  dest                660556 non-null  object        
 13  dest_city_name      660556 no

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


In [6]:
test_df.to_csv('flights_test.csv', index=False)

#### __Download Additional Datatset:  passengers__

In [8]:
passengers = pd.read_sql_query('select * from passengers',con=connection)

passengers.info()
passengers.head(3)

C:\Users\User\anaconda3\envs\jupyterlab-3.2.1\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350497 entries, 0 to 2350496
Data columns (total 38 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   departures_scheduled   float64
 1   departures_performed   float64
 2   payload                float64
 3   seats                  float64
 4   passengers             float64
 5   freight                float64
 6   mail                   float64
 7   distance               float64
 8   ramp_to_ramp           float64
 9   air_time               float64
 10  unique_carrier         object 
 11  airline_id             int64  
 12  unique_carrier_name    object 
 13  region                 object 
 14  carrier                object 
 15  carrier_name           object 
 16  carrier_group          int64  
 17  carrier_group_new      int64  
 18  origin_airport_id      int64  
 19  origin_city_market_id  int64  
 20  origin                 object 
 21  origin_city_name       object 
 22  origin_country    

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,0.0,2.0,4000.0,16.0,0.0,0.0,0.0,223.0,126.0,91.0,...,US,United States,6,651,1,2015,6,1,L,DU
1,0.0,1.0,2000.0,8.0,1.0,0.0,0.0,236.0,100.0,45.0,...,US,United States,6,651,1,2015,6,1,L,DU
2,0.0,1.0,3000.0,13.0,0.0,0.0,0.0,386.0,80.0,63.0,...,US,United States,6,641,1,2015,6,1,L,DU


In [9]:
passengers.to_csv('passengers.csv', index=False)

In [11]:
passengers_df = pd.read_csv('passengers.csv')
passengers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350497 entries, 0 to 2350496
Data columns (total 38 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   departures_scheduled   float64
 1   departures_performed   float64
 2   payload                float64
 3   seats                  float64
 4   passengers             float64
 5   freight                float64
 6   mail                   float64
 7   distance               float64
 8   ramp_to_ramp           float64
 9   air_time               float64
 10  unique_carrier         object 
 11  airline_id             int64  
 12  unique_carrier_name    object 
 13  region                 object 
 14  carrier                object 
 15  carrier_name           object 
 16  carrier_group          int64  
 17  carrier_group_new      int64  
 18  origin_airport_id      int64  
 19  origin_city_market_id  int64  
 20  origin                 object 
 21  origin_city_name       object 
 22  origin_country    

#### __Download Additional Datatset:  fuel_consumption__

In [16]:
connection = pg.connect("host='mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com' dbname=mid_term_project user=lhl_student password='lhl_student'")
fuel_df = pd.read_sql_query('select * from fuel_comsumption',con=connection)

fuel_df.info()
fuel_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3035 entries, 0 to 3034
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   month              3035 non-null   int64  
 1   airline_id         3034 non-null   float64
 2   unique_carrier     3034 non-null   object 
 3   carrier            3035 non-null   object 
 4   carrier_name       3034 non-null   object 
 5   carrier_group_new  3035 non-null   int64  
 6   sdomt_gallons      3034 non-null   float64
 7   satl_gallons       3034 non-null   float64
 8   spac_gallons       3034 non-null   float64
 9   slat_gallons       3033 non-null   float64
 10  sint_gallons       3034 non-null   float64
 11  ts_gallons         3034 non-null   float64
 12  tdomt_gallons      3032 non-null   float64
 13  tint_gallons       3034 non-null   float64
 14  total_gallons      3029 non-null   float64
 15  sdomt_cost         3035 non-null   float64
 16  satl_cost          3034 

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,NaN,None,0JQ,None,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5100.0,0.0,5100.0,2016
1,1,21352.0,0WQ,0WQ,Avjet Corporation,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,396216.0,140239.0,536455.0,2016
2,1,21645.0,23Q,23Q,Songbird Airways Inc.,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016


In [17]:
fuel_df.to_csv('fuel_consumption.csv', index=False)

### __Download Training Datatset:  flights__
> - __data from YR2019 where arr_delay > 0__
    - pdandas df: "flights2019y_df"
    - csv file name: "flights2019_delay"
    
> - __data from YR2019 where arr_delay <= 0__
    - pandas df: "flights2019n_df"
    - csv file name: "flights2019_no_delay"
    
> - __data from YR2018 where arr_delay > 0__
    - pandas df: "flights2018y_df"
    - csv file name: "flights2018_delay"
    
> - __data from YR2018 where arr_delay <= 0__
    - pandas df: "flights2018n_df"
    - csv file name: "flights2018_no_delay"

In [20]:
connection = pg.connect("host='mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com' dbname=mid_term_project user=lhl_student password='lhl_student'")
flights2019y_df = pd.read_sql_query("select * from flights where arr_delay > 0 and fl_date like '2019%'",con=connection)

flights2019y_df.info()
flights2019y_df.head(3)

C:\Users\User\anaconda3\envs\jupyterlab-3.2.1\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2737075 entries, 0 to 2737074
Data columns (total 42 columns):
 #   Column               Dtype  
---  ------               -----  
 0   fl_date              object 
 1   mkt_unique_carrier   object 
 2   branded_code_share   object 
 3   mkt_carrier          object 
 4   mkt_carrier_fl_num   int64  
 5   op_unique_carrier    object 
 6   tail_num             object 
 7   op_carrier_fl_num    int64  
 8   origin_airport_id    int64  
 9   origin               object 
 10  origin_city_name     object 
 11  dest_airport_id      int64  
 12  dest                 object 
 13  dest_city_name       object 
 14  crs_dep_time         int64  
 15  dep_time             float64
 16  dep_delay            float64
 17  taxi_out             float64
 18  wheels_off           float64
 19  wheels_on            float64
 20  taxi_in              float64
 21  crs_arr_time         int64  
 22  arr_time             float64
 23  arr_delay            float64
 24

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-05-01,UA,UA_CODESHARE,UA,3711,YX,N728YX,3711,10140,ABQ,...,744.0,32.0,0.0,0.0,0.0,0.0,747.0,19.0,19.0,None
1,2019-05-01,AA,AA_CODESHARE,AA,4553,YX,N114HQ,4553,11057,CLT,...,427.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
2,2019-05-01,AA,AA_CODESHARE,AA,4553,YX,N114HQ,4553,12339,IND,...,427.0,42.0,0.0,0.0,0.0,3.0,1711.0,22.0,22.0,None


In [21]:
flights2019y_df.to_csv('flights2019_delay.csv', index=False)

In [6]:
connection = pg.connect("host='mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com' dbname=mid_term_project user=lhl_student password='lhl_student'")
flights2019n_df = pd.read_sql_query("select * from flights where arr_delay <= 0 and fl_date like '2019%'",con=connection)

flights2019n_df.info()
flights2019n_df.head(3)

C:\Users\User\anaconda3\envs\jupyterlab-3.2.1\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5180189 entries, 0 to 5180188
Data columns (total 42 columns):
 #   Column               Dtype  
---  ------               -----  
 0   fl_date              object 
 1   mkt_unique_carrier   object 
 2   branded_code_share   object 
 3   mkt_carrier          object 
 4   mkt_carrier_fl_num   int64  
 5   op_unique_carrier    object 
 6   tail_num             object 
 7   op_carrier_fl_num    int64  
 8   origin_airport_id    int64  
 9   origin               object 
 10  origin_city_name     object 
 11  dest_airport_id      int64  
 12  dest                 object 
 13  dest_city_name       object 
 14  crs_dep_time         int64  
 15  dep_time             float64
 16  dep_delay            float64
 17  taxi_out             float64
 18  wheels_off           float64
 19  wheels_on            float64
 20  taxi_in              float64
 21  crs_arr_time         int64  
 22  arr_time             float64
 23  arr_delay            float64
 24

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-12-06,WN,WN,WN,543,WN,N926WN,543,11697,FLL,...,1092.0,None,None,None,None,None,None,NaN,NaN,None
1,2019-12-06,WN,WN,WN,648,WN,N8526W,648,11697,FLL,...,319.0,None,None,None,None,None,None,NaN,NaN,None
2,2019-12-06,WN,WN,WN,1507,WN,N7740A,1507,11697,FLL,...,319.0,None,None,None,None,None,None,NaN,NaN,None


In [7]:
flights2019n_df.to_csv('flights2019_no_delay.csv', index=False)

In [8]:
connection = pg.connect("host='mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com' dbname=mid_term_project user=lhl_student password='lhl_student'")
flights2018y_df = pd.read_sql_query("select * from flights where arr_delay > 0 and fl_date like '2018%'",con=connection)

flights2018y_df.info()
flights2018y_df.head(3)

C:\Users\User\anaconda3\envs\jupyterlab-3.2.1\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2738534 entries, 0 to 2738533
Data columns (total 42 columns):
 #   Column               Dtype  
---  ------               -----  
 0   fl_date              object 
 1   mkt_unique_carrier   object 
 2   branded_code_share   object 
 3   mkt_carrier          object 
 4   mkt_carrier_fl_num   int64  
 5   op_unique_carrier    object 
 6   tail_num             object 
 7   op_carrier_fl_num    int64  
 8   origin_airport_id    int64  
 9   origin               object 
 10  origin_city_name     object 
 11  dest_airport_id      int64  
 12  dest                 object 
 13  dest_city_name       object 
 14  crs_dep_time         int64  
 15  dep_time             float64
 16  dep_delay            float64
 17  taxi_out             float64
 18  wheels_off           float64
 19  wheels_on            float64
 20  taxi_in              float64
 21  crs_arr_time         int64  
 22  arr_time             float64
 23  arr_delay            float64
 24

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-03-09,B6,B6,B6,563,B6,N587JB,563,11697,FLL,...,1046.0,38.0,0.0,0.0,0.0,0.0,None,NaN,NaN,None
1,2018-03-09,B6,B6,B6,565,B6,N663JB,565,10721,BOS,...,1249.0,0.0,0.0,15.0,0.0,0.0,None,NaN,NaN,None
2,2018-03-09,B6,B6,B6,566,B6,N597JB,566,14843,SJU,...,1046.0,100.0,0.0,1.0,0.0,0.0,None,NaN,NaN,None


In [9]:
flights2018y_df.to_csv('flights2018_delay.csv', index=False)

#### __Due to the memory error, 4th training dataset, 'flights2018_no_delay' was imported through the separate Jupyter Notebook file, 'import_sql_table_v2.jpynb'.__

### __Reduce the training datasets__
> - __randomly resmapling 10% of the original training dataset as follows:__
     - __flights2019_delay:__ 2,737,075 rows -> 456,179 rows    =>  16.67% resampling
     - __flights2019_no delay:__ 5,180,189 rows -> 470,926 rows =>  9.09% resampling
     - __flights2018_delay:__ 2,738,534 rows -> 304,281 rows    => 11.11% resampling
     - __flights2018_no delay:__ 4,959,943 rows -> 330,662 rows =>  6.67% resampling

#### __flights2019_delay__

In [15]:
# Import the original train & test data
filename_train1 = 'flights2019_delay.csv'

# Reduce the original train & test datasets
#  - original train dataset = 558 MB -> reducing to 93 MB
n_train = sum(1 for line in open(filename_train1))-1  # Calculate number of rows in file
s_train = n_train//6  # sample size of 16.67%
skip_train = sorted(random.sample(range(1, n_train+1), n_train-s_train))  # n+1 to compensate for header 
reduced_train2019_delay = pd.read_csv(filename_train1, skiprows=skip_train)

reduced_train2019_delay.info()
reduced_train2019_delay.head(6)

# Export the reduced train & test datasets to csv files

filename_new_train1 = 'reduced1_flights2019_delay.csv'
reduced_train2019_delay.to_csv(filename_new_train1, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456179 entries, 0 to 456178
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   fl_date              456179 non-null  object 
 1   mkt_unique_carrier   456179 non-null  object 
 2   branded_code_share   456179 non-null  object 
 3   mkt_carrier          456179 non-null  object 
 4   mkt_carrier_fl_num   456179 non-null  int64  
 5   op_unique_carrier    456179 non-null  object 
 6   tail_num             456179 non-null  object 
 7   op_carrier_fl_num    456179 non-null  int64  
 8   origin_airport_id    456179 non-null  int64  
 9   origin               456179 non-null  object 
 10  origin_city_name     456179 non-null  object 
 11  dest_airport_id      456179 non-null  int64  
 12  dest                 456179 non-null  object 
 13  dest_city_name       456179 non-null  object 
 14  crs_dep_time         456179 non-null  int64  
 15  dep_time         

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-05-01,AA,AA_CODESHARE,AA,4593,YX,N436YX,4593,13851,OKC,...,1158.0,1.0,0.0,0.0,0.0,98.0,NaN,NaN,NaN,NaN
1,2019-05-01,AA,AA_CODESHARE,AA,4646,YX,N102HQ,4646,13244,MEM,...,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-05-01,AA,AA_CODESHARE,AA,4646,YX,N102HQ,4646,14100,PHL,...,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-05-01,AA,AA_CODESHARE,AA,4648,YX,N110HQ,4648,11624,EYW,...,736.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-05-01,AA,AA_CODESHARE,AA,4656,YX,N114HQ,4656,10868,CAE,...,88.0,2.0,0.0,0.0,0.0,13.0,NaN,NaN,NaN,NaN
5,2019-05-01,AA,AA_CODESHARE,AA,4681,YX,N103HQ,4681,12339,IND,...,588.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### __flights2019_no delay__

In [18]:
# Import the original train & test data
filename_train2 = 'flights2019_no_delay.csv'

# Reduce the original train & test datasets
#  - original train dataset = 1,008 MB -> reducing to 92 MB
n_train = sum(1 for line in open(filename_train2))-1  # Calculate number of rows in file
s_train = n_train//11  # sample size of 9.09%
skip_train = sorted(random.sample(range(1, n_train+1), n_train-s_train))  # n+1 to compensate for header 
reduced_train2019_nodelay = pd.read_csv(filename_train2, skiprows=skip_train)

reduced_train2019_nodelay.info()
reduced_train2019_nodelay.head(6)

# Export the reduced train & test datasets to csv files

filename_new_train2 = 'reduced1_flights2019_nodelay.csv'
reduced_train2019_nodelay.to_csv(filename_new_train2, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470926 entries, 0 to 470925
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   fl_date              470926 non-null  object 
 1   mkt_unique_carrier   470926 non-null  object 
 2   branded_code_share   470926 non-null  object 
 3   mkt_carrier          470926 non-null  object 
 4   mkt_carrier_fl_num   470926 non-null  int64  
 5   op_unique_carrier    470926 non-null  object 
 6   tail_num             470926 non-null  object 
 7   op_carrier_fl_num    470926 non-null  int64  
 8   origin_airport_id    470926 non-null  int64  
 9   origin               470926 non-null  object 
 10  origin_city_name     470926 non-null  object 
 11  dest_airport_id      470926 non-null  int64  
 12  dest                 470926 non-null  object 
 13  dest_city_name       470926 non-null  object 
 14  crs_dep_time         470926 non-null  int64  
 15  dep_time         

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-12-06,WN,WN,WN,1064,WN,N432WN,1064,11697,FLL,...,177.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-06,WN,WN,WN,2549,WN,N264LV,2549,11697,FLL,...,992.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-12-06,WN,WN,WN,765,WN,N249WN,765,11697,FLL,...,1188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-12-06,WN,WN,WN,2178,WN,N475WN,2178,11697,FLL,...,1145.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-06,WN,WN,WN,513,WN,N7742B,513,11697,FLL,...,197.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2019-12-06,WN,WN,WN,2603,WN,N730SW,2603,11996,GSP,...,153.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### __flights2018_delay__

In [19]:
# Import the original train & test data
filename_train3 = 'flights2018_delay.csv'

# Reduce the original train & test datasets
#  - original train dataset = 557 MB -> reducing to 62 MB
n_train = sum(1 for line in open(filename_train3))-1  # Calculate number of rows in file
s_train = n_train//9  # sample size of 11.11%
skip_train = sorted(random.sample(range(1, n_train+1), n_train-s_train))  # n+1 to compensate for header 
reduced_train2018_delay = pd.read_csv(filename_train3, skiprows=skip_train)

reduced_train2018_delay.info()
reduced_train2018_delay.head(6)

# Export the reduced train & test datasets to csv files

filename_new_train3 = 'reduced1_flights2018_delay.csv'
reduced_train2018_delay.to_csv(filename_new_train3, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304281 entries, 0 to 304280
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   fl_date              304281 non-null  object 
 1   mkt_unique_carrier   304281 non-null  object 
 2   branded_code_share   304281 non-null  object 
 3   mkt_carrier          304281 non-null  object 
 4   mkt_carrier_fl_num   304281 non-null  int64  
 5   op_unique_carrier    304281 non-null  object 
 6   tail_num             304281 non-null  object 
 7   op_carrier_fl_num    304281 non-null  int64  
 8   origin_airport_id    304281 non-null  int64  
 9   origin               304281 non-null  object 
 10  origin_city_name     304281 non-null  object 
 11  dest_airport_id      304281 non-null  int64  
 12  dest                 304281 non-null  object 
 13  dest_city_name       304281 non-null  object 
 14  crs_dep_time         304281 non-null  int64  
 15  dep_time         

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-03-09,B6,B6,B6,567,B6,N661JB,567,10397,ATL,...,581.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-03-09,B6,B6,B6,653,B6,N632JB,653,12478,JFK,...,1028.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-03-09,B6,B6,B6,711,B6,N969JB,711,12478,JFK,...,2248.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-03-09,B6,B6,B6,811,B6,N537JB,811,11618,EWR,...,1068.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-03-09,B6,B6,B6,859,B6,N265JB,859,10721,BOS,...,280.0,12.0,0.0,21.0,0.0,9.0,NaN,NaN,NaN,NaN
5,2018-03-09,B6,B6,B6,918,B6,N283JB,918,12478,JFK,...,187.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### __flights2018_no delay__

In [22]:
# Import the original train & test data
filename_train4 = 'flights2018_no_delay.csv'

# Reduce the original train & test datasets
#  - original train dataset = 965 MB -> reducing to 96 MB
n_train = sum(1 for line in open(filename_train4))-1  # Calculate number of rows in file
s_train = n_train//15  # sample size of 6.67%
skip_train = sorted(random.sample(range(1, n_train+1), n_train-s_train))  # n+1 to compensate for header 
reduced_train2018_nodelay = pd.read_csv(filename_train4, skiprows=skip_train)

reduced_train2018_nodelay.info()
reduced_train2018_nodelay.head(6)

# Export the reduced train & test datasets to csv files

filename_new_train4 = 'reduced1_flights2018_nodelay.csv'
reduced_train2018_nodelay.to_csv(filename_new_train4, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330662 entries, 0 to 330661
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   fl_date              330662 non-null  object 
 1   mkt_unique_carrier   330662 non-null  object 
 2   branded_code_share   330662 non-null  object 
 3   mkt_carrier          330662 non-null  object 
 4   mkt_carrier_fl_num   330662 non-null  int64  
 5   op_unique_carrier    330662 non-null  object 
 6   tail_num             330662 non-null  object 
 7   op_carrier_fl_num    330662 non-null  int64  
 8   origin_airport_id    330662 non-null  int64  
 9   origin               330662 non-null  object 
 10  origin_city_name     330662 non-null  object 
 11  dest_airport_id      330662 non-null  int64  
 12  dest                 330662 non-null  object 
 13  dest_city_name       330662 non-null  object 
 14  crs_dep_time         330662 non-null  int64  
 15  dep_time         

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-03-01,B6,B6,B6,232,B6,N635JB,232,12954,LGB,...,588.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-03-01,B6,B6,B6,264,B6,N558JB,264,14747,SEA,...,2422.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-03-01,B6,B6,B6,269,B6,N627JB,269,10721,BOS,...,1237.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-03-01,B6,B6,B6,275,B6,N657JB,275,10529,BDL,...,1666.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-03-01,B6,B6,B6,282,B6,N505JB,282,12191,HOU,...,1428.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2018-03-01,B6,B6,B6,499,B6,N517JB,499,12953,LGA,...,950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### __Combine four reduced training datasets above to one training dataset__

#### Combine the following dataframes:
> - reduced_train2019_delay
> - reduced_train2019_nodelay
> - reduced_train2018_delay
> - reduced_train2018_nodelay

In [23]:
# combine all four reduced datasets
frames = [reduced_train2019_delay, reduced_train2019_nodelay, 
          reduced_train2018_delay, reduced_train2018_nodelay]

train_df = pd.concat(frames)

train_df.info()
train_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1562048 entries, 0 to 330661
Data columns (total 42 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   fl_date              1562048 non-null  object 
 1   mkt_unique_carrier   1562048 non-null  object 
 2   branded_code_share   1562048 non-null  object 
 3   mkt_carrier          1562048 non-null  object 
 4   mkt_carrier_fl_num   1562048 non-null  int64  
 5   op_unique_carrier    1562048 non-null  object 
 6   tail_num             1562048 non-null  object 
 7   op_carrier_fl_num    1562048 non-null  int64  
 8   origin_airport_id    1562048 non-null  int64  
 9   origin               1562048 non-null  object 
 10  origin_city_name     1562048 non-null  object 
 11  dest_airport_id      1562048 non-null  int64  
 12  dest                 1562048 non-null  object 
 13  dest_city_name       1562048 non-null  object 
 14  crs_dep_time         1562048 non-null  int64  
 15 

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-05-01,AA,AA_CODESHARE,AA,4593,YX,N436YX,4593,13851,OKC,...,1158.0,1.0,0.0,0.0,0.0,98.0,NaN,NaN,NaN,NaN
1,2019-05-01,AA,AA_CODESHARE,AA,4646,YX,N102HQ,4646,13244,MEM,...,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-05-01,AA,AA_CODESHARE,AA,4646,YX,N102HQ,4646,14100,PHL,...,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-05-01,AA,AA_CODESHARE,AA,4648,YX,N110HQ,4648,11624,EYW,...,736.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-05-01,AA,AA_CODESHARE,AA,4656,YX,N114HQ,4656,10868,CAE,...,88.0,2.0,0.0,0.0,0.0,13.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330657,2018-11-30,UA,UA_CODESHARE,UA,6140,YV,N85320,6140,12266,IAH,...,374.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330658,2018-11-30,UA,UA_CODESHARE,UA,6005,YV,N87345,6005,12264,IAD,...,384.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330659,2018-11-30,UA,UA_CODESHARE,UA,6131,YV,N503MJ,6131,12264,IAD,...,441.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330660,2018-11-30,UA,UA_CODESHARE,UA,6045,YV,N511MJ,6045,13931,ORF,...,157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# export the combined train_df to csv
train_df.to_csv('combined_reduced_flights.csv', index=False)

In [28]:
# check if the data are balanced.

train_df['target_class'] = np.where(train_df['arr_delay'] > 0, 1, 0)
train_df['target_class'].value_counts()

print(f"\nDelayed Data: {len(train_df[train_df['target_class'] == 1])} |\
 {len(train_df[train_df['target_class'] == 1])*100/len(train_df)}%")
print(f"Non-delayed Data: {len(train_df[train_df['target_class'] == 0])} |\
 {len(train_df[train_df['target_class'] == 0])*100/len(train_df)}%")

0    801588
1    760460
Name: target_class, dtype: int64


Delayed Data: 760460 | 48.6835231695825%
Non-delayed Data: 801588 | 51.3164768304175%


In [30]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1562048 entries, 0 to 330661
Data columns (total 43 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   fl_date              1562048 non-null  object 
 1   mkt_unique_carrier   1562048 non-null  object 
 2   branded_code_share   1562048 non-null  object 
 3   mkt_carrier          1562048 non-null  object 
 4   mkt_carrier_fl_num   1562048 non-null  int64  
 5   op_unique_carrier    1562048 non-null  object 
 6   tail_num             1562048 non-null  object 
 7   op_carrier_fl_num    1562048 non-null  int64  
 8   origin_airport_id    1562048 non-null  int64  
 9   origin               1562048 non-null  object 
 10  origin_city_name     1562048 non-null  object 
 11  dest_airport_id      1562048 non-null  int64  
 12  dest                 1562048 non-null  object 
 13  dest_city_name       1562048 non-null  object 
 14  crs_dep_time         1562048 non-null  int64  
 15 

In [29]:
# export the combined train_df to csv
train_df.to_csv('combined_reduced_flights.csv', index=False)